In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By

In [2]:
def get_jobs(num_jobs, verbose):
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''

    # Initializing the webdriver
    options = webdriver.ChromeOptions()

    # Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')

    # Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Users/Brian/Documents/salary_proj/chromedriver", options=options)
    driver.set_window_size(1120, 1000)

#     url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
#     url = 'https://www.glassdoor.com/Job/data-scientist-jobs-SRCH_KO0,14.htm?'
    page = 1
    url = f"https://www.glassdoor.com/Job/data-scientist-jobs-SRCH_KO0,14_IP{page}.htm"

    driver.get(url)
    jobs = []

    def check_prompt():
        try:
            driver.find_element(By.CLASS_NAME, "modal_main").click()

        except:
            pass

        time.sleep(.1)

        try:
            driver.find_element(By.CLASS_NAME, "modal_closeIcon").click()  # clicking to the X.
        except:
            pass

    def page_loaded():
        try:
            driver.find_element(By.CLASS_NAME, "css-17bh0pp")
            return False
        except:
            return True

    while len(jobs) < num_jobs:  # If true, should be still looking for new jobs.

        # Let the page load. Change this number based on your internet speed.
        # Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(5)
#         page_jobs = 0
        print('Going through each job in this page')
        job_buttons = driver.find_elements(By.CLASS_NAME,
                                           "react-job-listing")  # jl for Job Listing. These are the buttons we're going to click.
        print(len(job_buttons))
        if len(job_buttons) == 0:
            break
        for job_button in job_buttons:
            check_prompt()
            

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
#             if page_jobs >= 2:
#                 break

            job_button.click()  # You might
            
            time.sleep(5)
            if not page_loaded():
                continue
            check_prompt()
            
            time.sleep(5)
            collected_successfully = False
            #             find_element(By.XPATH, "xpath")

            while not collected_successfully:
                try:
                    print("Trying to collect information")
                    # company_name = driver.find_element(By.XPATH, './/div[@class="employerName"]').text
                    company_name = (driver.find_element(By.CLASS_NAME, 'css-xuk5ye').text).split('\n')[0]
                    print(company_name)
                    location = driver.find_element(By.CLASS_NAME, 'css-56kyx5').text
                    job_title = driver.find_element(By.CLASS_NAME, 'css-1j389vi').text
                    job_description = driver.find_element(By.CLASS_NAME, 'jobDescriptionContent').text
                    collected_successfully = True
                except:
                    print("No information collected successfully")
                    status = page_loaded()
                    if not status:
                        break
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element(By.CLASS_NAME, 'css-y2jiyn').text
            except NoSuchElementException:
                salary_estimate = -1  # You need to set a "not found value. It's important."

            try:

#                 rating = driver.find_element(By.CLASS_NAME, 'css-y2jiyn').text
                rating = (driver.find_element(By.CLASS_NAME, 'css-xuk5ye').text).split('\n')[1]
            except:
                rating = -1  # You need to set a "not found value. It's important."

            # Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            # Going to the Company tab...
            # clicking on this:
            # <div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                time.sleep(5)
                details = driver.find_elements(By.CLASS_NAME, 'css-1ff36h2')
                headquarters = -1
                size = details[1].text
                founded = details[2].text
                type_of_ownership = details[3].text
                industry = details[4].text
                sector = details[5].text
                revenue = details[6].text
                competitors = -1
            
            except:  # Rarely, some job postings do not have the "Company" tab.
                print("Details Not found")
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1


            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title": job_title,
                         "Salary Estimate": salary_estimate,
                         "Job Description": job_title,
                         "Rating": rating,
                         "Company Name": company_name,
                         "Location": location,
                         "Headquarters": location,
                         "Size": size,
                         "Founded": founded,
                         "Type of ownership": type_of_ownership,
                         "Industry": industry,
                         "Sector": sector,
                         "Revenue": revenue,
                         "Competitors": competitors})
#             page_jobs = page_jobs + 1
            # add job to jobs

            
        # Clicking on the "next page" button
        if len(jobs) < num_jobs:
            try:
                page = page+1
                url = f"https://www.glassdoor.com/Job/data-scientist-jobs-SRCH_KO0,14_IP{page}.htm"
                driver.get(url)
#                 nav_buttons = driver.find_elements(By.CLASS_NAME, "navIcon-svg")
#                 next_button = nav_buttons[1]
#                 next_button.click()
    #             driver.find_element(By.XPATH, './/li[@class="next"]//a').click()
            except NoSuchElementException:
                print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs,
                                                                                                             len(jobs)))
        else:
            break
  

    return pd.DataFrame(jobs)  # This line converts the dictionary object into a pandas DataFrame.



In [3]:
df = get_jobs(2000, False)

C:\Users\Brian\AppData\Local\Temp\ipykernel_10684\3204107243.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/Users/Brian/Documents/salary_proj/chromedriver", options=options)


Going through each job in this page
30
Progress: 0/2000
Trying to collect information
Myticas Consulting
Details Not found
Progress: 1/2000
Trying to collect information
GTA (Global Technology Associates)
Details Not found
Progress: 2/2000
Trying to collect information
3G Federal Solutions, LLC
Details Not found
Progress: 3/2000
Trying to collect information
Cilable
Details Not found
Progress: 4/2000
Trying to collect information
Augray
Progress: 5/2000
Trying to collect information
Mint-Technologies
Details Not found
Progress: 6/2000
Trying to collect information
REVOLVE
Progress: 7/2000
Trying to collect information
Net2Aspire
Details Not found
Progress: 8/2000
Trying to collect information
Audible
Progress: 9/2000
Trying to collect information
Apptrics LLC
Details Not found
Progress: 10/2000
Trying to collect information
Murmuration
Details Not found
Progress: 11/2000
Trying to collect information
Acima
Progress: 12/2000
Trying to collect information
Business Intelli Solutions
Detai

Trying to collect information
UT Southwestern Medical Center
Progress: 99/2000
Trying to collect information
Layer3
Progress: 100/2000
Trying to collect information
Murmuration
Details Not found
Progress: 101/2000
Trying to collect information
Bell Federal Systems Inc.
Details Not found
Progress: 102/2000
Trying to collect information
Los Angeles County Department of Human Resources
Details Not found
Progress: 103/2000
Trying to collect information
3G Federal Solutions, LLC
Details Not found
Progress: 104/2000
Trying to collect information
CVS Health
Progress: 105/2000
Trying to collect information
Acima
Progress: 106/2000
Trying to collect information
REVOLVE
Progress: 107/2000
Trying to collect information
Dataworks
Progress: 108/2000
Trying to collect information
Flipside Crypto
Details Not found
Progress: 109/2000
Trying to collect information
User Interviews
Details Not found
Progress: 110/2000
Trying to collect information
Activision
Progress: 111/2000
Trying to collect informati

Progress: 212/2000
Trying to collect information
BDSA
Details Not found
Progress: 213/2000
Trying to collect information
Acima
Progress: 214/2000
Trying to collect information
Starbucks
Progress: 215/2000
Trying to collect information
T. Rowe Price
Progress: 216/2000
Trying to collect information
Layer3
Progress: 217/2000
Trying to collect information
United Nations
Progress: 218/2000
Trying to collect information
PepsiCo
Progress: 219/2000
Trying to collect information
Roq.ad GmbH
Details Not found
Progress: 220/2000
Trying to collect information
L'Oreal
Progress: 221/2000
Trying to collect information
Yelp
Progress: 222/2000
Trying to collect information
Mint-Technologies
Details Not found
Progress: 223/2000
Trying to collect information
Codecademy
Progress: 224/2000
Trying to collect information
Digital Trike
Details Not found
Progress: 225/2000
Trying to collect information
Electric Power Research Institute
Details Not found
Progress: 226/2000
Trying to collect information
centrifu

Progress: 324/2000
Trying to collect information
Radian
Progress: 325/2000
Trying to collect information
centrifuge llc
Details Not found
Progress: 326/2000
Trying to collect information
Activision
Progress: 327/2000
Trying to collect information
State Farm
Going through each job in this page
30
Progress: 328/2000
Trying to collect information
Spotify
Progress: 329/2000
Trying to collect information
BDSA
Details Not found
Progress: 330/2000
Trying to collect information
UT Southwestern Medical Center
Progress: 331/2000
Trying to collect information
Augray
Progress: 332/2000
Trying to collect information
Acima
Progress: 333/2000
Trying to collect information
PepsiCo
Progress: 334/2000
Trying to collect information
Starbucks
Progress: 335/2000
Trying to collect information
Digital Trike
Details Not found
Progress: 336/2000
Trying to collect information
Spotify
Progress: 337/2000
Trying to collect information
Zelus Analytics
Details Not found
Progress: 338/2000
Trying to collect informati

In [ ]:
df

In [9]:
df.to_csv('2000jobs.csv')

In [10]:
df.head(20)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,BHJOB15656_16794 - Data Scientist,-1,BHJOB15656_16794 - Data Scientist,3.7,Myticas Consulting,Remote,Remote,-1,-1,-1,-1,-1,-1,-1
1,Data Scientist,$85.00 /hr (est.),Data Scientist,4.3,Infologitech,Remote,Remote,-1,-1,-1,-1,-1,-1,-1
2,Data Scientist III,"$80,000 /yr (est.)",Data Scientist III,4.4,GTA (Global Technology Associates),"Englewood, CO","Englewood, CO",-1,-1,-1,-1,-1,-1,-1
3,Data Scientist,"$115,000 /yr (est.)",Data Scientist,-1,"3G Federal Solutions, LLC",Remote,Remote,-1,-1,-1,-1,-1,-1,-1
4,Junior Data Scientist,"$100,000 /yr (est.)",Junior Data Scientist,-1,Talentheed Inc,Remote,Remote,-1,-1,-1,-1,-1,-1,-1
5,Data Scientist,$77.50 /hr (est.),Data Scientist,4.6,Kani Solutions,Remote,Remote,201 to 500 Employees,2009,Company - Private,Information Technology Support Services,Information Technology,$5 to $10 million (USD),-1
6,Data Scientist,$87.50 /hr (est.),Data Scientist,5.0,Tricon IT Solutions (For End Clients),"Orlando, FL","Orlando, FL",-1,-1,-1,-1,-1,-1,-1
7,Data Scientist - Personalization,"$85,503 /yr (est.)",Data Scientist - Personalization,4.0,Audible,"Newark, NJ","Newark, NJ",1001 to 5000 Employees,1995,Subsidiary or Business Segment,Film Production,Media & Communication,Unknown / Non-Applicable,-1
8,Data Scientist,"$120,000 /yr (est.)",Data Scientist,5.0,Murmuration,"New York, NY","New York, NY",-1,-1,-1,-1,-1,-1,-1
9,Data Scientist,$75.00 /hr (est.),Data Scientist,-1,Mint-Technologies,"Richmond, VA","Richmond, VA",-1,-1,-1,-1,-1,-1,-1
